In [76]:
#Questions
    #What to do with unbalanced data
    #How to align test and train data from feature engineering

In [77]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2


In [78]:
from fastai.structured import *
from fastai.column_data import *
from sklearn.metrics import roc_auc_score
from pandas_summary import DataFrameSummary

np.set_printoptions(threshold = 50, edgeitems = 20)

PATH = 'data/'

In [79]:
ls {PATH}

application_test.csv                installments_payments.csv
application_test.csv.zip            installments_payments.csv.zip
application_train.csv               models/
application_train.csv.zip           POS_CASH_balance.csv
bureau_balance.csv                  POS_CASH_balance.csv.zip
bureau_balance.csv.zip              previous_application.csv
bureau.csv                          previous_application.csv.zip
bureau.csv.zip                      sample_submission.csv
credit_card_balance.csv             sample_submission.csv.zip
credit_card_balance.csv.zip         subm/
df_pref                             tmp/
HomeCredit_columns_description.csv


In [80]:
#data = pd.read_feather('tmp/data')
data = pd.read_csv(PATH + 'application_train.csv')
data.shape

(307511, 122)

In [81]:
#data_test = pd.read_feather('tmp/data_test')
data_test = pd.read_csv(PATH + 'application_test.csv')
data_test.shape

(48744, 121)

In [7]:
# nrow = 1000
#application_train = pd.read_csv(path + 'application_train.csv', nrow = 1000)
#bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
#bureau = pd.read_csv(path + 'bureau.csv')
#credit_card_balance = pd.read_csv(path + 'credit_card_balance.csv')
#installments_payments = pd.read_csv(path + 'installments_payments.csv')
#pos_cash_balance = pd.read_csv(path+'POS_CASH_balance.csv')
#previous_application = pd.read_csv(path + 'previous_application.csv')

In [82]:
cat_vars = list(data.select_dtypes('object').columns); len(cat_vars)


16

In [83]:
contin_vars = list(data.select_dtypes(np.number).columns); len(contin_vars)

106

In [84]:
contin_vars.remove('SK_ID_CURR'); len(contin_vars)

105

In [85]:
contin_vars.remove('TARGET'); len(contin_vars)

104

In [86]:
dep = 'TARGET'
data = data[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()
# data = data[cat_vars+contin_vars].copy()
data.shape

(307511, 122)

In [87]:
dep = 'TARGET'
data_test[dep] = 0
data_test = data_test[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()
# data_test = data_test[cat_vars+contin_vars+[dep]].copy()
data_test.shape

(48744, 122)

In [25]:
#!pip install pandas-summary

In [88]:
def display_allRows(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 10):
            display(df)

In [75]:
display_allRows(data.dtypes)

NAME_CONTRACT_TYPE              category
CODE_GENDER                     category
FLAG_OWN_CAR                    category
FLAG_OWN_REALTY                 category
NAME_TYPE_SUITE                 category
NAME_INCOME_TYPE                category
NAME_EDUCATION_TYPE             category
NAME_FAMILY_STATUS              category
NAME_HOUSING_TYPE               category
OCCUPATION_TYPE                 category
WEEKDAY_APPR_PROCESS_START      category
ORGANIZATION_TYPE               category
FONDKAPREMONT_MODE              category
HOUSETYPE_MODE                  category
WALLSMATERIAL_MODE              category
EMERGENCYSTATE_MODE             category
CNT_CHILDREN                       int64
AMT_INCOME_TOTAL                 float64
AMT_CREDIT                       float64
AMT_ANNUITY                      float64
AMT_GOODS_PRICE                  float64
REGION_POPULATION_RELATIVE       float64
DAYS_BIRTH                         int64
DAYS_EMPLOYED                      int64
DAYS_REGISTRATIO

In [16]:
data_test.head()

NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY NAME_TYPE_SUITE  \
0         Cash loans           F            N               Y   Unaccompanied   
1         Cash loans           M            N               Y   Unaccompanied   
2         Cash loans           M            Y               Y             NaN   
3         Cash loans           F            N               Y   Unaccompanied   
4         Cash loans           M            Y               N   Unaccompanied   

  NAME_INCOME_TYPE            NAME_EDUCATION_TYPE NAME_FAMILY_STATUS  \
0          Working               Higher education            Married   
1          Working  Secondary / secondary special            Married   
2          Working               Higher education            Married   
3          Working  Secondary / secondary special            Married   
4          Working  Secondary / secondary special            Married   

   NAME_HOUSING_TYPE     OCCUPATION_TYPE   ...   FLAG_DOCUMENT_19  \
0  House / apartment                 NaN   ...                  0   
1  House / apartment  Low-skill Laborers   ...                  0   
2  House / apartment             Drivers   ...                  0   
3  House / apartment         Sales staff   ...                  0   
4  House / apartment                 NaN   ...                  0   

  FLAG_DOCUMENT_20 FLAG_DOCUMENT_21 AMT_REQ_CREDIT_BUREAU_HOUR  \
0                0                0                        0.0   
1                0                0                        0.0   
2                0                0                        0.0   
3                0                0                        0.0   
4                0                0                        NaN   

  AMT_REQ_CREDIT_BUREAU_DAY AMT_REQ_CREDIT_BUREAU_WEEK  \
0                       0.0                        0.0   
1                       0.0                        0.0   
2                       0.0                        0.0   
3                       0.0                        0.0   
4                       NaN                        NaN   

   AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
0                        0.0                        0.0   
1                        0.0                        0.0   
2                        0.0                        1.0   
3                        0.0                        0.0   
4                        NaN                        NaN   

   AMT_REQ_CREDIT_BUREAU_YEAR  TARGET  
0                         0.0       0  
1                         3.0       0  
2                         4.0       0  
3                         3.0       0  
4                         NaN       0  

[5 rows x 122 columns]

In [89]:
for v in cat_vars: data[v] = data[v].astype('category').cat.as_ordered()

In [90]:
data.shape

(307511, 122)

In [15]:
#data.select_dtypes('category')

NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0              Cash loans           M            N               Y   
1              Cash loans           F            N               N   
2         Revolving loans           M            Y               Y   
3              Cash loans           F            N               Y   
4              Cash loans           M            N               Y   
5              Cash loans           M            N               Y   
6              Cash loans           F            Y               Y   
7              Cash loans           M            Y               Y   
8              Cash loans           F            N               Y   
9         Revolving loans           M            N               Y   
10             Cash loans           F            N               Y   
11             Cash loans           F            N               Y   
12             Cash loans           F            N               Y   
13             Cash loans           M            Y               N   
14             Cash loans           F            N               Y   
15             Cash loans           M            Y               Y   
16             Cash loans           M            N               N   
17        Revolving loans           F            N               Y   
18        Revolving loans           F            N               Y   
19             Cash loans           F            N               Y   
20        Revolving loans           M            Y               Y   
21             Cash loans           F            Y               Y   
22             Cash loans           F            N               N   
23             Cash loans           F            N               Y   
24             Cash loans           M            Y               N   
25             Cash loans           F            N               Y   
26             Cash loans           F            N               Y   
27             Cash loans           M            N               Y   
28             Cash loans           M            Y               Y   
29        Revolving loans           M            N               Y   
...                   ...         ...          ...             ...   
307481         Cash loans           M            N               Y   
307482         Cash loans           F            Y               Y   
307483         Cash loans           F            N               Y   
307484         Cash loans           F            Y               N   
307485    Revolving loans           F            N               Y   
307486         Cash loans           F            Y               Y   
307487         Cash loans           M            N               Y   
307488         Cash loans           F            N               N   
307489         Cash loans           F            N               Y   
307490         Cash loans           M            N               Y   
307491         Cash loans           M            Y               Y   
307492         Cash loans           M            Y               Y   
307493         Cash loans           F            N               Y   
307494         Cash loans           M            Y               N   
307495         Cash loans           M            Y               N   
307496    Revolving loans           F            N               N   
307497         Cash loans           F            Y               Y   
307498         Cash loans           M            Y               Y   
307499         Cash loans           F            N               Y   
307500         Cash loans           F            N               Y   
307501         Cash loans           F            N               Y   
307502         Cash loans           F            N               Y   
307503         Cash loans           F            N               Y   
307504         Cash loans           F            N               Y   
307505         Cash loans           F            N               Y   
307506         Ca

In [24]:
#data_test.dtypes

NAME_CONTRACT_TYPE            category
CODE_GENDER                   category
FLAG_OWN_CAR                  category
FLAG_OWN_REALTY               category
NAME_TYPE_SUITE               category
NAME_INCOME_TYPE              category
NAME_EDUCATION_TYPE           category
NAME_FAMILY_STATUS            category
NAME_HOUSING_TYPE             category
OCCUPATION_TYPE               category
WEEKDAY_APPR_PROCESS_START    category
ORGANIZATION_TYPE             category
FONDKAPREMONT_MODE            category
HOUSETYPE_MODE                category
WALLSMATERIAL_MODE            category
EMERGENCYSTATE_MODE           category
SK_ID_CURR                       int64
TARGET                           int64
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
AMT_GOODS_PRICE                float64
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED            

In [86]:
#train_cats(data)

In [91]:
apply_cats(data_test, data)

In [92]:
for v in contin_vars:
    data[v] = data[v].fillna(0).astype('float32')
    data_test[v] = data_test[v].fillna(0).astype('float32')

In [93]:
data.shape, data_test.shape

((307511, 122), (48744, 122))

In [94]:
data.isnull()

NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0                    False        False         False            False   
1                    False        False         False            False   
2                    False        False         False            False   
3                    False        False         False            False   
4                    False        False         False            False   
5                    False        False         False            False   
6                    False        False         False            False   
7                    False        False         False            False   
8                    False        False         False            False   
9                    False        False         False            False   
10                   False        False         False            False   
11                   False        False         False            False   
12                   False        False         False            False   
13                   False        False         False            False   
14                   False        False         False            False   
15                   False        False         False            False   
16                   False        False         False            False   
17                   False        False         False            False   
18                   False        False         False            False   
19                   False        False         False            False   
20                   False        False         False            False   
21                   False        False         False            False   
22                   False        False         False            False   
23                   False        False         False            False   
24                   False        False         False            False   
25                   False        False         False            False   
26                   False        False         False            False   
27                   False        False         False            False   
28                   False        False         False            False   
29                   False        False         False            False   
...                    ...          ...           ...              ...   
307481               False        False         False            False   
307482               False        False         False            False   
307483               False        False         False            False   
307484               False        False         False            False   
307485               False        False         False            False   
307486               False        False         False            False   
307487               False        False         False            False   
307488               False        False         False            False   
307489               False        False         False            False   
307490               False        False         False            False   
307491               False        False         False            False   
307492               False        False         False            False   
307493               False        False         False            False   
307494               False        False         False            False   
307495               False        False         False            False   
307496               False        False         False            False   
307497               False        False         False            False   
307498               False        False         False            False   
307499               False        False         False            False   
307500               False        False         False            False   
307501               False        False         False            False   
307502               False        False         False            False   
307503      

# Deep Learning

Merics

In [95]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [roc_auc_score(y_train, m.predict(X_train)), roc_auc_score(y_valid, m.predict(X_valid)),
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m,'oob_score_'): res.append(m.oob_score_)
    print(res)
    


Run on a sample

In [96]:
n = len(data)
idxs = get_cv_idxs(n, val_pct = 50000/n)
data_samp = data.iloc[idxs].set_index('SK_ID_CURR')
samp_size = len(data_samp); samp_size

50000

Run on full set

In [97]:
sample_size = len(data)
data_samp = data.set_index('SK_ID_CURR')
data_samp.shape

(307511, 121)

In [98]:
data_samp.head()

NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
SK_ID_CURR                                                               
100002             Cash loans           M            N               Y   
100003             Cash loans           F            N               N   
100004        Revolving loans           M            Y               Y   
100006             Cash loans           F            N               Y   
100007             Cash loans           M            N               Y   

           NAME_TYPE_SUITE NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
SK_ID_CURR                                                                   
100002       Unaccompanied          Working  Secondary / secondary special   
100003              Family    State servant               Higher education   
100004       Unaccompanied          Working  Secondary / secondary special   
100006       Unaccompanied          Working  Secondary / secondary special   
100007       Unaccompanied          Working  Secondary / secondary special   

              NAME_FAMILY_STATUS  NAME_HOUSING_TYPE OCCUPATION_TYPE   ...    \
SK_ID_CURR                                                            ...     
100002      Single / not married  House / apartment        Laborers   ...     
100003                   Married  House / apartment      Core staff   ...     
100004      Single / not married  House / apartment        Laborers   ...     
100006            Civil marriage  House / apartment        Laborers   ...     
100007      Single / not married  House / apartment      Core staff   ...     

           FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21  \
SK_ID_CURR                                                      
100002                  0.0              0.0              0.0   
100003                  0.0              0.0              0.0   
100004                  0.0              0.0              0.0   
100006                  0.0              0.0              0.0   
100007                  0.0              0.0              0.0   

           AMT_REQ_CREDIT_BUREAU_HOUR AMT_REQ_CREDIT_BUREAU_DAY  \
SK_ID_CURR                                                        
100002                            0.0                       0.0   
100003                            0.0                       0.0   
100004                            0.0                       0.0   
100006                            0.0                       0.0   
100007                            0.0                       0.0   

           AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
SK_ID_CURR                                                         
100002                            0.0                        0.0   
100003                            0.0                        0.0   
100004                            0.0                        0.0   
100006                            0.0                        0.0   
100007                            0.0                        0.0   

            AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  TARGET  
SK_ID_CURR                                                                 
100002                            0.0                         1.0       1  
100003                            0.0                         0.0       0  
100004                            0.0                         0.0       0  
100006                            0.0                         0.0       0  
100007                            0.0                         0.0       0  

[5 rows x 121 columns]

ModelData object

In [99]:
df, y, nas, mapper = proc_df(data_samp, 'TARGET', do_scale=True)

In [100]:
#data_test['TARGET'] = 0 #??

In [101]:
df_test, _, nas, mapper = proc_df(data_test, 'TARGET', do_scale=True, 
                    skip_flds=['SK_ID_CURR'], mapper=mapper, na_dict=nas)

In [102]:
#validation index
train_ratio = 0.75
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df))); len(val_idx)

270011

In [103]:
#ModelData object
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y, 
                                       cat_flds = cat_vars, bs=128, test_df=df_test )

In [104]:
#show cardinality for each categories
cat_sz = [(c, len(data_samp[c].cat.categories)+1) for c in cat_vars]

In [105]:
cat_sz

[('NAME_CONTRACT_TYPE', 3),
 ('CODE_GENDER', 4),
 ('FLAG_OWN_CAR', 3),
 ('FLAG_OWN_REALTY', 3),
 ('NAME_TYPE_SUITE', 8),
 ('NAME_INCOME_TYPE', 9),
 ('NAME_EDUCATION_TYPE', 6),
 ('NAME_FAMILY_STATUS', 7),
 ('NAME_HOUSING_TYPE', 7),
 ('OCCUPATION_TYPE', 19),
 ('WEEKDAY_APPR_PROCESS_START', 8),
 ('ORGANIZATION_TYPE', 59),
 ('FONDKAPREMONT_MODE', 5),
 ('HOUSETYPE_MODE', 4),
 ('WALLSMATERIAL_MODE', 8),
 ('EMERGENCYSTATE_MODE', 3)]

In [106]:
#set embeding sizes
emb_szs = [(c, min(50,(c+1)//2)) for _, c in cat_sz]

In [107]:
emb_szs

[(3, 2),
 (4, 2),
 (3, 2),
 (3, 2),
 (8, 4),
 (9, 5),
 (6, 3),
 (7, 4),
 (7, 4),
 (19, 10),
 (8, 4),
 (59, 30),
 (5, 3),
 (4, 2),
 (8, 4),
 (3, 2)]

In [108]:
m = md.get_learner(emb_szs, len(data.columns)-len(cat_vars),
                  0.04, 1, [1000, 500], [0.001, 0.01])
lr = 1e-3

In [109]:
m.lr_find()

  0%|          | 0/293 [00:00<?, ?it/s]


RuntimeError: running_mean should contain 104 elements not 106

In [62]:
m.fit(lr, 3, metrics=[exp_rmspe])

NameError: name 'exp_rmspe' is not defined